In [1]:
import sys
sys.path.append('../')
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from utils import Doc, read
from attribute import TFIDFKeywordExtractor
import numpy as np

Model loaded succeed


In [2]:
# dev_lyrics = read()[:500]
# Doc.load_corpus(dev_lyrics)
Doc.load()

In [3]:
symbols = ['PAD', 'SOS', 'EOS']
Doc.extend_vocab(symbols)
vocab = Doc.get_vocab()
print(len(vocab))
for symbol in symbols:
    print(vocab[symbol])
print(Doc.idxs_to_text(vocab['PAD']))

10003
10000
10001
10002
PAD


In [4]:
list_of_list_of_lines = [doc.get_lines() for doc in Doc.get_corpus()]
text_sample = list_of_list_of_lines
# print(Doc.idxs_to_text(text_sample[0]))
keywords = TFIDFKeywordExtractor(text_sample, len(Doc.get_vocab()),n_keywords = 1)
keywords.get_keywords()
# pairs = list(zip([i for j in keywords.get_keywords() for i in j],\
#             [i for j in text_sample for i in j]))
pairs = list(zip([i for j in text_sample  for i in j],\
            [i for j in text_sample for i in j]))


hidden_size = 512
teacher_forcing_ratio = 0.5
MIN_LENGTH = 0
MAX_LENGTH = 40
batch_size = 32

def keywords_filterPair(p):
    return MIN_LENGTH < len(p[0]) < MAX_LENGTH and \
        MIN_LENGTH < len(p[1]) < MAX_LENGTH

def keywords_filterPairs(pairs):
    return [pair for pair in pairs if keywords_filterPair(pair)]
filtered_pairs = keywords_filterPairs(pairs)

In [5]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [6]:
from sklearn.cross_validation import train_test_split
train_data,test_data, _, __ =  \
    train_test_split(filtered_pairs,np.zeros(np.array(filtered_pairs).shape),test_size=0.2, random_state=0)
    

/home/jinxingyu/anaconda2/envs/lyrics/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# train_input = [x[0].tolist() for x in train_data]
train_input = [x[0].tolist()+[vocab["EOS"]] for x in train_data]
train_label = [x[1].tolist()+[vocab["EOS"]] for x in train_data]

# test_input = [x[0].tolist() for x in test_data]
test_input = [x[0].tolist()+[vocab["EOS"]] for x in test_data]
test_label = [x[1].tolist()+[vocab["EOS"]] for x in test_data]

In [8]:
train_input = [x+[vocab['PAD']] * (MAX_LENGTH - len(x)) for x in train_input]
train_label = [x+[vocab['PAD']] * (MAX_LENGTH - len(x)) for x in train_label]

test_input = [x+[vocab['PAD']] * (MAX_LENGTH - len(x)) for x in test_input]
test_label = [x+[vocab['PAD']] * (MAX_LENGTH - len(x)) for x in test_label]

In [9]:
# train_input = np.array(train_input)[:,:,np.newaxis]
# train_label = np.array(train_label)[:,:,np.newaxis]
# test_input = np.array(test_input)[:,:,np.newaxis]
# test_label = np.array(test_label)[:,:,np.newaxis]
from keras.utils import to_categorical
train_input = np.array(train_input)
train_label = (np.array(train_label)[:,:,np.newaxis])
# test_input = np.array(test_input)
# test_label = np.array(to_categorical(test_label))



Using TensorFlow backend.
/home/jinxingyu/anaconda2/envs/lyrics/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [10]:
from random import randint
from keras.utils import to_categorical
def generate_batch_data_random(x, y, batch_size):
    ylen = len(y)
    loopcount = ylen // batch_size
    i = randint(0,loopcount)
    print('1')
    return np.array((x[i * batch_size:(i + 1) * batch_size])),\
    np.array(to_categorical(y[i * batch_size:(i + 1) * batch_size]))

In [11]:
x,y = generate_batch_data_random(train_input, train_label, 100)

1


In [12]:
x.shape

(100, 40)

In [13]:
import random
np.array([random.choice(x)]).shape

(1, 40)

In [14]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import RepeatVector, TimeDistributed, Activation
from keras.layers.core import Dense
from seq2seq.cells import LSTMDecoderCell, AttentionDecoderCell
import time
import numpy as np
import re
from functools import reduce 
import random
import bisect

In [15]:
def weight_choice(weight):
    weight_sum = []
    sum = 0
    for a in weight:
        sum += a
        weight_sum.append(sum)
    t = random.uniform(0, sum)
    return bisect.bisect_right(weight_sum, t)

In [16]:
weight_choice([0.1,0.9])

1

In [ ]:
!export CUDA_VISIBLE_DEVICES=1

In [ ]:
vocab_size = len(vocab)  # keras进行embedding的时候必须进行len(vocab)+1
input_maxlen = MAX_LENGTH
tar_maxlen = MAX_LENGTH
output_dim = vocab_size
hidden_dim = 512



# decoder_mode = 3  # 0 最简单模式，1 [1]向后模式，2 [2] Peek模式，3 [3]Attention模式
# if decoder_mode == 3:
#     encoder_top_layer = LSTM(hidden_dim, return_sequences=True)
# else:
#     encoder_top_layer = LSTM(hidden_dim)

# if decoder_mode == 0:
#     decoder_top_layer = LSTM(hidden_dim, return_sequences=True)
#     decoder_top_layer.get_weights()
# elif decoder_mode == 1:
#     decoder_top_layer = LSTMDecoderCell(hidden_dim=hidden_dim, output_dim=hidden_dim
#                                    )
# elif decoder_mode == 2:
#     decoder_top_layer = LSTMDecoderCell(hidden_dim=hidden_dim, output_dim=hidden_dim
#                                    )
# elif decoder_mode == 3:
#     decoder_top_layer = AttentionDecoderCell(hidden_dim=hidden_dim, output_dim=hidden_dim
#                                         )

en_de_model = Sequential()
en_de_model.add(Embedding(input_dim=vocab_size,
                          output_dim=hidden_dim,
                          input_length=input_maxlen))
# en_de_model.add(encoder_top_layer)
# if decoder_mode == 0:
#     en_de_model.add(RepeatVector(tar_maxlen))
# en_de_model.add(decoder_top_layer)
# from seq2seq.models import Seq2Seq
# en_de_model.add(Seq2Seq(input_dim=hidden_dim, hidden_dim=hidden_dim, output_length=MAX_LENGTH, \
#                         output_dim=output_dim,teacher_force=False))
en_de_model.add(LSTM(hidden_dim,input_shape = [None,None,hidden_dim], return_sequences=True))
en_de_model.add(TimeDistributed(Dense(output_dim)))
en_de_model.add(Activation('softmax'))




print('Compiling...')
time_start = time.time()
en_de_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
time_end = time.time()
print('Compiled, cost time:%fsecond!' % (time_end - time_start))
for iter_num in range(5000):
    en_de_model.fit(x,y, batch_size=batch_size, epochs=1)
    test_sentence = np.array([random.choice(x)])
    out_predicts = en_de_model.predict(test_sentence)
    for i_idx, out_predict in enumerate(out_predicts):
        predict_sequence = []
        for predict_vector in out_predict:
            next_index = np.argmax(predict_vector)
#             next_index = weight_choice(predict_vector) # predict_vector is output_dim(here is 51) dimension vector
#             import pdb
#             pdb.set_trace()
            next_token = Doc.idxs_to_text(int(next_index)) # key of idx_to_word starts from 1, but next_index can be 0(the probability is very low!)
            predict_sequence.append(next_token)

        print('Target output:', Doc.idxs_to_text(test_sentence[i_idx]))
        print('Predict output:', predict_sequence)
    print('Current iter_num is:%d' % iter_num)

Compiling...
Compiled, cost time:0.027453second!
Epoch 1/1
100/100 [==============================] - 2s 21ms/step - loss: 7.7482
Target output: 连西单也不孤单EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:0
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 1.7036
Target output: 至少今天我还活着EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD

100/100 [==============================] - 1s 8ms/step - loss: 0.8499
Target output: 我的世界因你一点一滴倾倒EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '我', '你', '你', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:16
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 0.8751
Target output: 男恭喜啊恭喜啊恭喜你EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '我', '的', '的', '的', 'EOS', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PA

100/100 [==============================] - 1s 8ms/step - loss: 0.5526
Target output: 你应该明白一场的梦EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '爱', '爱', '的', '的', '的', '歌', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:32
Epoch 1/1
100/100 [==============================] - 1s 9ms/step - loss: 0.5458
Target output: 如果我说是如果EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '我', '爱', '我', '我', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Curre

100/100 [==============================] - 1s 8ms/step - loss: 0.3284
Target output: 就算我走了也没人会想我念我的EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '我', '快活', '有', '有', '钱', '么', '人', '让', '我', '念', '我', '的', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:48
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 0.3239
Target output: 你就像翩翩彩蝶的化身EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['你', '你', '像', '翩翩', '躺', '的', '人', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:49
Epoch 1/1
10

100/100 [==============================] - 1s 8ms/step - loss: 0.1412
Target output: 你就像翩翩彩蝶的化身EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['你', '就', '像', '翩翩', '彩蝶', '的', '化身', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:64
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 0.1345
Target output: 找到了安定EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['时间', '了', '安定', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD'

...

100/100 [==============================] - 1s 9ms/step - loss: 1.7731e-07
Target output: 愿为你守候从日落到清晨EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['愿', '为', '你', '守候', '从', '日落', '到', '清晨', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:288
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 1.7665e-07
Target output: 洛丽塔就像爱情烟花EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['洛丽塔', '就', '像', '爱情', '烟花', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD

100/100 [==============================] - 1s 9ms/step - loss: 1.6841e-07
Target output: 我的世界因你一点一滴倾倒EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '的', '世界', '因', '你', '一点一滴', '倾倒', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:304
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 1.6795e-07
Target output: 是不是我的家EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['是', '不', '是', '我', '的', '家', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', '

100/100 [==============================] - 1s 8ms/step - loss: 1.6116e-07
Target output: 歌手怀旧EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['歌手', '怀旧', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:320
Epoch 1/1
100/100 [==============================] - 1s 9ms/step - loss: 1.6078e-07
Target output: 我在我幻想的世界EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['我', '在', '我', '幻想', '的', '世界', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD

100/100 [==============================] - 1s 8ms/step - loss: 1.5537e-07
Target output: 找到了安定EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['找到', '了', '安定', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:336
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 1.5499e-07
Target output: 做最快乐的自我EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['做', '最', '快乐', '的', '自我', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD

100/100 [==============================] - 1s 8ms/step - loss: 1.5062e-07
Target output: 好想把你留住EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['好', '想', '把', '你', '留住', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Current iter_num is:352
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 1.5038e-07
Target output: 女最后再让我看看你EOSPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
Predict output: ['女', '最后', '再', '让', '我', '看看', '你', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', '

In [ ]:
# import seq2seq
# from seq2seq.models import SimpleSeq2Seq

# batch_size = 32
# epochs = 10

# model = SimpleSeq2Seq(input_dim = len(vocab), hidden_dim = 512, output_dim= len(vocab),output_length=40)
# # model.load_weights("word2seq.h5")
# model.compile(loss='mse', optimizer='rmsprop')
# model.fit_generator(generate_batch_data_random(train_input[:round(0.9*len(train_input))], \
#                                                train_label[:round(0.9*len(train_label))], batch_size),                                                      
#     samples_per_epoch=0.9*len(train_input)//batch_size*batch_size,
#     nb_epoch=epochs, 
#     validation_data=generate_batch_data_random(train_input[round(0.9*len(train_input)),:],\
#                                          train_label[round(0.9*len(train_label)),:] ,batch_size),
#     nb_val_samples=(0.1*len(train_input)//batch_size*batch_size))

In [ ]:
# import seq2seq
# from seq2seq.models import SimpleSeq2Seq

# batch_size = 32
# epochs = 10
# model = SimpleSeq2Seq(input_dim = 1, hidden_dim = 512, output_dim= len(vocab),output_length=40)
# for i in range(10):
#     # model.load_weights("word2seq.h5")
#     model.compile(loss='mse', optimizer='rmsprop')
#     batch_input,batch_label = generate_batch_data_random(train_input, \
#                                            train_label, batch_size)

#     model.fit(batch_input,batch_label,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2)

In [ ]:
# import seq2seq
# from seq2seq.models import SimpleSeq2Seq

# batch_size = 32
# epochs = 10
# model = SimpleSeq2Seq(input_dim = 1, hidden_dim = 512, output_dim= 1000,output_length=40)

In [ ]:
# import seq2seq
# from seq2seq.models import SimpleSeq2Seq

# batch_size = 32
# epochs = 10

# model = SimpleSeq2Seq(input_dim = 1, hidden_dim = 512, output_dim= 1,output_length=40)
# # model.load_weights("word2seq.h5")
# model.compile(loss='mse', optimizer='rmsprop')
# model.fit(train_input, train_label,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)

In [ ]:
# import keras
# import random
# # my_rms = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-06)
# # model.compile(loss='mse', optimizer='adam')
# # model.fit(train_input, train_label,
# #         batch_size=batch_size,
# #       epochs=epochs,
# #       validation_split=0.2)

# test_single_input = random.choice(test_input)[np.newaxis,:,:]
# print('test single input ',Doc.idxs_to_text(test_single_input[0,:,0]))
# print('test single output',Doc.idxs_to_text(model.predict(test_single_input)[0,:,0]))